# Generating the Blind Set

* Also known as **holdout dataset**

* Cross-validation is not sufficient tot estimate unbiased generalization performance.
    - Model hyper-parameters are still optimized on the training set through cross-valiation and grid-search
    - This may lead to some degree of overfitting on training data
    - Using a blind set helps us to generate a 'never-seen-before condition'
    
## Generation Criteria:

* Structures deposited **after** January 2015
     - Release year of JPred4 is 2014
* X-ray crystals with resolution < 2,5 $\overset{\circ}{A}$
* Chain lenght in the range of 50 - 300 residues
* Advanced search -> Entry Polymer Types:
    - Protein OR Protein/NA 
* All pairs of sequences within the blind set should share less than 30% sequence identity ('internal redundancy'):
- By using ```blastclust``` we can reduce the redundancy

* When comparing sequences of the blindset with the JPRED set: All pairs (blind - Jpred) have an less than 30% identity  ('external redundancy')
- This will be ensured using ```blastp```

* The final blind set will comprise 150 proteins which will be randomly selected among those that meet the above criteria

### 1. Downloading Data from the PDB

Checked the boxes "Entry ID","Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)".

[here the link to my search](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22greater%22%2C%22negation%22%3Afalse%2C%22value%22%3A%222015-01-31T00%3A00%3A00Z%22%2C%22attribute%22%3A%22rcsb_accession_info.deposit_date%22%7D%2C%22node_id%22%3A0%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22X-RAY%20DIFFRACTION%22%2C%22attribute%22%3A%22exptl.method%22%7D%2C%22node_id%22%3A1%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22less_or_equal%22%2C%22negation%22%3Afalse%2C%22value%22%3A2.5%2C%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%7D%2C%22node_id%22%3A2%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22or%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22Protein%20(only)%22%2C%22attribute%22%3A%22rcsb_entry_info.selected_polymer_entity_types%22%7D%2C%22node_id%22%3A3%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22Protein%2FNA%22%2C%22attribute%22%3A%22rcsb_entry_info.selected_polymer_entity_types%22%7D%2C%22node_id%22%3A4%7D%5D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22operator%22%3A%22range_closed%22%2C%22negation%22%3Afalse%2C%22value%22%3A%5B50%2C300%5D%2C%22attribute%22%3A%22entity_poly.rcsb_sample_sequence_length%22%7D%2C%22node_id%22%3A5%7D%5D%7D%5D%2C%22label%22%3A%22text%22%7D%5D%2C%22label%22%3A%22query-builder%22%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A100%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%2C%22request_info%22%3A%7B%22src%22%3A%22ui%22%2C%22query_id%22%3A%22dc19df09287d4c5a80018000a03e2a6d%22%7D%7D)

* Downloaded as CSV 

For some reason it automatically adds "Entry ID" as column 1. Whenever there is another chain of the same ID the first line of col 1 will be empty --> removed it using awk:

In [3]:
head -6 1.csv 

"Entry ID","Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)",
"5EV3","GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV3",
,"UUUUUUUU","NA-hybrid","B","5EV3",
"5EV2","GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV2",
,"UUUUUUUU","NA-hybrid","B","5EV2",
"5EV4","GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV4",


In [5]:
for i in {1..5}
do
    cat ${i}.csv | awk '{sub(/[^,]*/,"");sub(/,/,"")} 1' > ${i}new.csv 
done

In [1]:
head -5 4new.csv #now it looks like this

"Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)",
"MALVDGFLELERSSGKLEWSAILQKMASDLGFSKILFGLLPKDSQDYENAFIVGNYPAAWREHYDRAGYARVDPTVSHCTQSVLPIFWEPSIYQTRKQHEFFEEASAAGLVYGLTMPLHGARGELGALSLSVEAENRAEANRFMESVLPTLWMLKDYALQSGAGLAFEHPVSKPVVLTSREKEVLQWCAIGKTSWEISVICNCSEANVNFHMGNIRRKFGVTSRRVAAIMAVNLGLITL","Protein","A, B","6MWL",
"SVAHGLAWSYYIGYLRLILPELQARIRTYNQHYNNLLRGAVSQRLYILLPLDCGVPDNLSMADPNIRFLDKLPQQTADRAGIKDRVYSNSIYELLENGQRAGTCVLEYATPLQTLFAMSQYSQAGFSREDRLEQAKLFCQTLEDILADAPESQNNCRLIAYQEPADDSSFSLSQEVLRHLRQEEKEEV","Protein","A, B","6MX0",
"MKLVWTLSSWDDYEFWQRTDARMVEKINDLIRNAKRTPFAGLGKPEPLKGDMAGYWSRRITAEHRFVYRVSGSGSEQRLEVIQCRFHYQLEVLFQ","Protein","A, B","6N90",
"MDYKDDDDKGSLVPRGSHMYLRITNIVESSFFTKFIIYLIVLNGITMGLETSKTFMQSFGVYTTLFNQIVITIFTIEIILRIYVHRISFFKDPWSLFDFFVVAISLVPTSSGFEILRVLRVLRLFRLVTAVPQMRKIVSALISVIPGMLSVIALMTLFFYIFAIMATQLFGERFPEWFGTLGESFYTLFQVMTLESWSMGIVRPLMEVYPYAWVFFIPFIFVVTFVMINLVVAIIVDAMAILNQKEEQHIIDEVQSH","Protein","B","6MWA",


### 2. Parsing and Filtering

First I need to be aware that some of the chains are nucleic acids ("NA-hybrid").

Remove all lines containing
*  "NA-hybrid" 
*  "DNA"
*  "RNA"

to obtain protein sequences only!

```  sed -n '/Protein/p' ./filename 


In [10]:
# Removing all lines that do NOT contain "Protein" --> this removes the header too!
head -1 1new.csv > aa_only.csv # Adding correct header to top of file that will be appended all "Protein" lines

for i in {1..5} # Appending only lines containing word "Protein"
do
    sed -n '/Protein/p' ${i}new.csv >> aa_only.csv 
done

In [11]:
head aa_only.csv

"Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV3",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV2",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV4",
"SLRSDLINALYDENQKYDVCGIISAEGKIYPLGSDTKVLSTIFELFSRPIINKIAEKHGYIVEEPKQQNHYPDFTLYKPSEPNKKIAIDIKTTYTNKENEKIKFTLGGYTSFIRNNTKNIVYPFDQYIAHWIIGYVYTRVATRKSSLKTYNINELNEIPKPYKGVKVFLQDKWVIAGDLAGSGNTTNIGSIHAHYKDFVEGKGIFDSEDEFLDYWRNYERTSQLRNDKYNNISEYR

In [2]:
grep "Protein" aa_only.csv | wc -l #  29638 Protein chains in the set

   29638


### Switched to python 3 kernel:

In [1]:
# Loading relevant packages:

import pandas as pd
import numpy as np
import seaborn as sns
# sns.set() #do we really need that


In [4]:
df_aa = pd.read_csv("aa_only.csv")
df_aa

,Sequence,Entity Polymer Type,Chain ID,Entry Id (Polymer Entity Identifiers),Unnamed: 4
0,GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPV...,Protein,A,5EV3,NaN
1,GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPV...,Protein,A,5EV2,NaN
2,GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPV...,Protein,A,5EV4,NaN
3,SLRSDLINALYDENQKYDVCGIISAEGKIYPLGSDTKVLSTIFELF...,Protein,"A, B",5F8A,NaN
4,MAVLAESELGSEAQRERRKRILDATMAIASKGGYEAVQMRAVADRA...,Protein,"A, B",5FMP,NaN
...,...,...,...,...,...
29633,DVLMTQIPLSLPVSLGDQASISCRSSQNIVHSNGNTYLEWYLQKPG...,Protein,"B, L",6SF6,NaN
29634,PSPCHEKADCILERDGSRS,Protein,"C, D",6SF6,NaN
29635,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,Protein,A,6SET,NaN
29636,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,Protein,A,6SEW,NaN


In [5]:
# Where did the unnamed come from??
unnamed = df_aa["Unnamed: 4"] # Maybe trailing comma?
unnamed.unique()

array([nan])

#### Finding unique values in cloumn "Entity Polymer Type"
I want to find unique values as described [here](https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/)

This way I can be sure that my cleaning was successful

In [6]:
# Finding unique names in cols
pol_type = df_aa['Entity Polymer Type']
pol_type.unique() # I am now sure that all DNA and RNA and Protein/NA lines have been removed.

array(['Protein'], dtype=object)

In [12]:
!head -3 aa_only.csv

"Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV3",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV2",


In [13]:
!sed '1d' aa_only.csv > noheader_aa_only.csv # removing header before generating fasta


In [17]:
!wc -l aa_only.csv

   29639 aa_only.csv


In [18]:
!wc -l noheader_aa_only.csv # value matches file is ok proceeding to make fasta

   29638 noheader_aa_only.csv


#### I want to consider the comma as a field sepparator 

Since there are several chains per entry denoted as e.g.:

``` "CAGTTTCAAACTC","D, I","5FD3",```

I need to remove the comma between the chains first and replace it with a space:
*  ``` sed 's/\, / /g' $i.csv ```

In [12]:
# Removing commas between chains

sed 's/\, //g' aa_only.csv > nospace.csv

head nospace.csv

"Sequence","Entity Polymer Type","Chain ID","Entry Id (Polymer Entity Identifiers)",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV3",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV2",
"GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN","Protein","A","5EV4",
"SLRSDLINALYDENQKYDVCGIISAEGKIYPLGSDTKVLSTIFELFSRPIINKIAEKHGYIVEEPKQQNHYPDFTLYKPSEPNKKIAIDIKTTYTNKENEKIKFTLGGYTSFIRNNTKNIVYPFDQYIAHWIIGYVYTRVATRKSSLKTYNINELNEIPKPYKGVKVFLQDKWVIAGDLAGSGNTTNIGSIHAHYKDFVEGKGIFDSEDEFLDYWRNYERTSQLRNDKYNNISEYR

## 3. Generating the FASTA file

* Using awk: defining the comma as field sepparator.

* ``` awk -F ',' ```

* filtering for lenght in the range of 50 - 300 

*  ``` 'length($1) > 50 && length($1) < 301 {print ">"$4":"$3"\n"$1}' ```

In [13]:
cat nospace.csv | awk -F ',' 'length($1) > 50 && length($1) < 301 {print ">"$4":"$3"\n"$1}' | sed 's/\"//g' > 50_300.fasta

In [14]:
head 50_300.fasta

>5EV3:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5EV2:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5EV4:A
GSQMTRQARRLYVGNIPFGITEEAMMDFFNAQMRLGGLTQAPGNPVLAVQINQDKNFAFLEFRSVDETTQAMAFDGIIFQGQSLKIRRPHDYQPLPGMSENPSVYVPGVVSTVVPDSAHKLFIGGLPNYLNDDQVKELLTSFGPLKAFNLVKDSATGLSKGYAFCEYVDINVTDQAIAGLNGMQLGDKKLLVQRASVGAKN
>5F8A:AB
SLRSDLINALYDENQKYDVCGIISAEGKIYPLGSDTKVLSTIFELFSRPIINKIAEKHGYIVEEPKQQNHYPDFTLYKPSEPNKKIAIDIKTTYTNKENEKIKFTLGGYTSFIRNNTKNIVYPFDQYIAHWIIGYVYTRVATRKSSLKTYNINELNEIPKPYKGVKVFLQDKWVIAGDLAGSGNTTNIGSIHAHYKDFVEGKGIFDSEDEFLDYWRNYERTSQLRNDKYNNISEYRNWIYRGRK
>5FMP:AB
MAVLAESELGSEAQRERRKRILDATMAIASKGGYEAVQMRAVADRADVAVGTLYRYFPSKVHLLVSALGREFSRIDAKTDRSAVAGATPFQRLNFMVGKLNRAMQRN

#### Sequences containing X need to be removed:

Working on the script and testing it along the way. The final result is saved as

#### removeX.py


In [ ]:
def lines_to_list(filename):
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(filename, "r") as rfile:
        content_list = rfile.readlines()
        return content_list

myfastalist = lines_to_list("50_300.fasta")  #works

In [51]:
len(myfastalist)

59276

In [109]:
def split_list(liste):
    ''' Splits a evennumbered list into two lists. id_list contains all odd items while seq_list contains all even items. Returns the two lists.'''
    id_list = liste[::2]
    seq_list = liste[1::2]
    return id_list, seq_list

# teste = ['a', 'b', 'c', 'd', 'e', 'f'] #Works
ids, seq = split_list(myfastalist)


print(len(ids))
print(len(seq))
print(len(myfastalist))

29638
29638
59276


In [105]:
def remove_X(id1, seq2):
    '''Removes items containing X in the sequence list but also the ID in the ID list. 
    Returns an ID list and an'''
    noXid = []
    noXseq = []
    for i in range(len(id1)):
        flag = "X" in seq2[i]
        if flag == False:
            noXid.append(id1[i])
            noXseq.append(seq2[i])
    return noXid, noXseq


In [128]:
new_id, new_seq = remove_X(ids, seq)

print(len(new_id))
print(len(new_seq))

def no_X_id_and_seq(id_list, seq_list):
    ''' Joins the lists to a big list containing both id and sequences. Returns one big list'''
    biglist = []
    for i in range(len(id_list)):
        biglist.append(id_list[i])
        biglist.append(seq_list[i])
    return biglist


28951
28951


In [130]:
biglist = no_X_id_and_seq(new_id, new_seq)
len(biglist)

57902

In [132]:
#Writing list to file:

def list_to_fasta(liste):
    '''Takes one id list and one sequlist as input. Writes all elements i to 
    a file. Returns the file.'''
    with open('no_X.fasta', 'w') as F:
        for i in liste:
            F.write(str(i))
    F.close            
    

In [133]:
list_to_fasta(biglist)

In [136]:
def filter_short(infile, outfile):
    del_seq_index = []
    lines_list = []
    with open(infile) as rfile:
        lines_list = rfile.readlines()
        for i in range(1, len(lines_list),2):
            if len(lines_list[i]) < 7:
                del_seq_index.append(i-1) # appending header index
                del_seq_index.append(i)   # appending sequence index
    with open(outfile, 'w') as wfile:
        for i in range(len(lines_list)):
            if i in del_seq_index:
                continue
            wfile.write(lines_list[i])    
        
filter_short("no_X.fasta", 'longsequences.fasta')                   

### 4. Clustering the Sequences in blastclust

Sending file to be clustered to the VM:

### 5. Picking longest seuqence of each cluster: by default col1

cat final_clusters | awk $1 {print} > best_of_cluster

In [1]:
for i in {1..5}
do
    mv $i.csv ./orignial_csv/$i.csv
done

In [2]:
for i in {1..5}
do
     mv ${i}new.csv ./orignial_csv/${i}new.csv
done     

#### 6. Generating FASTA Containing ONLY Sequences of Best Cluster

* I think the easiest is if I generate a list of best_of_final_cluster

    - Need to first generate a new file that contains the ">" character in front of every sequence


* And generate a dictionary of the no_X_long_sequs.fasta   

* Then I want to use the list to loop on the dictionary

In [4]:
# making sure the output of the script will be fasta standard
cat best_of_final_cluster | sed 's/^/>/' > crocodile_ids_final_cluster

In [ ]:
#!/usr/bin/env python3
import sys

def lines_to_list(infile1): 
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(infile1, "r") as rfile:
        content_list = rfile.readlines()
        return content_list

def split_list(infile1):
    ''' Splits a evennumbered list into two lists. id_list contains 
    all odd items while seq_list contains all even items. Returns the two lists.'''
    myfastalist = lines_to_list(infile1)  #works
    id_list = myfastalist[::2]
    seq_list = myfastalist[1::2]
    return id_list, seq_list

def dict_from_lists(infile1):
    '''Takes feeds two lists into a dictionary. 
    Returns the dicitonary'''
    id_list, seq_list = split_list(infile1)
    keys = id_list
    values = seq_list
    full_dict = dict(zip(keys, values))
    return full_dict
    
def keep_whats_in_dict(infile1, infile2, outfile):
    '''Loops through a list and a dictionary. Appending the values
    of the list (PDB ids which are also the keys of the dictionary) and the
    values of the dictionary to the outfile.'''
    idlist = lines_to_list(infile2) # reading ids from file into list
    aa_dict = dict_from_lists(infile1)
    with open(outfile, 'a') as afile:
        for i in idlist:
            afile.write(i) #appending ID in even lines
            afile.write(aa_dict[i]) # appending value (sequ) in odd lines

if __name__ == '__main__':
    infile1 = sys.argv[1]
    infile2 = sys.argv[2]
    outfile = sys.argv[3]
    keep_whats_in_dict(infile1, infile2, outfile)


In [5]:
# run the script
python  make_fasta_from_best_of_each_cluster.py no_X_long_sequs.fasta crocodile_ids_final_cluster crocodile_best_of_final_cluster.fasta


In [1]:
scp -i ~/.ssh/id_rsa.pub ./make_fasta_from_best_of_each_cluster.py proj:~/lb2-2020-project-englander

make_fasta_from_best_of_each_cluster.py       100% 1536    19.3KB/s   00:00    


In [6]:
scp -i ~/.ssh/id_rsa.pub ./crocodile_best_of_final_cluster.fasta proj:~/lb2-2020-project-englander

crocodile_best_of_final_cluster.fasta         100%  704KB   1.1MB/s   00:00    


### 6. Mergeing all fasta files of the jpred set 

Need it later to generate blastdb.

In [4]:
cat *.fasta > JPREDall.fasta  # merging

In [8]:
grep ">" JPREDall.fasta | wc -l  # works --> merged all 1348 files.

    1348


In [9]:
scp -i ~/.ssh/id_rsa.pub ./JPREDall.fasta proj:~/lb2-2020-project-englander

JPREDall.fasta                                100%  226KB 777.5KB/s   00:00    


### 7. Reducing Redundancy

I want to produce a blind testset that is as dissimilar to the training set as possible.

* Running blastp with blindset against JPRED training set
* I dentifying all sequences in the blindset that have LESS than 30% seq ID with any other sequ in the training set.



In [8]:
# copying hits.blast.tab to local
scp -i ~/.ssh/um19_id_rsa um19@m19.lsb.biocomp.unibo.it:~/lb2-2020-project-englander/makeblastdb/hits.blastp.tab ~/01-Unibo/02_Lab2/project_blindset/

hits.blastp.tab                               100%   43KB 177.0KB/s   00:00    


In [9]:
wc -l hits.blastp.tab

     759 hits.blastp.tab


In [10]:
head -4 hits.blastp.tab
# $1 pdb_Id $2 jprd id $3 % identity

5MA4:A	d4duia_	83.439	157	26	0	3	159	7	163	1.87e-86	251
5MA4:A	d4duia_	77.444	133	29	1	165	296	31	163	5.71e-63	191
6D8X:A	d1fcya_	29.032	186	126	3	109	292	53	234	1.41e-19	80.5
6G4T:A	d3k34a_	56.757	259	111	1	7	265	1	258	2.30e-110	314


## Generating Non-Redundant Set With Least Similarty 

Step 3 from the Slides: "Filter out from the preliminary chain set, all chains having at least one BLAST hit with SI >= 30% with any sequence in the JPRED4 dataset

### Generating 2 Files Containing Only Relevant Lines

file above30:
* I'll extract ```$1``` if ``` $3 > 30 ```

file below30
* I have to extract ```$1``` if col ```$3 < 30 ```


In [13]:
# awk keep filed if col 3 < 30 pipe to new file.
awk -F ' ' '$3 < 30 {print $1 " " $3}' hits.blastp.tab > below_30.hits
awk -F ' ' '$3 >= 30 {print $1 " " $3}' hits.blastp.tab > above_30.hits

#### Keeping IDs only

In [16]:
awk -F ' ' '$3 < 30 {print $1}' hits.blastp.tab > id_below_30.hits
awk -F ' ' '$3 >= 30 {print $1}' hits.blastp.tab > id_above_30.hits

In [13]:
def lines_to_list(infile1): 
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(infile1, "r") as rfile:
        content_list = rfile.readlines()
        return content_list
        
below = lines_to_list("id_below_30.hits")  # list of all ids scoring below 30% id
above = lines_to_list('id_above_30.hits')  # list of all ids scoring above and equal to 30% id



In [33]:
print(len(below))
print(len(above))

187
572


In [35]:
# "6D8X:A\n" in above
# head -30 id_below_30.hits > ids_test
# head -30 id_above_30.hits > ids_above_test

def remove_matches(lower, higher):
    '''Takes two lists as input and returns a list that contains
    all values of 'lower' values that are NOT element of 'higher'.'''
    keepers = []          # list holding all ids that have not scored >= 30% with any of the JPRED sequnces
    for i in lower:       
        if i not in higher: 
            keepers.append(i)  # keeps only ids that are not reported in the list "above"
    return keepers

keep = remove_matches(below, above) 

print(len(keep))
# Have 177 unique sequnces with no match above 30% id with any other sequ in the testing (JPRED) set.

177


In [34]:
all_ids = lines_to_list("best_of_final_cluster") # generating list of all ids that were in the blastp input
print(len(all_ids))

3920


In [30]:
def keep_mis_matches(biglist, partiallist):
    '''Stores exclusively biglist values that are not reported in partiallist.
    Returns the biglist with all partiallist matches removed. Keeps all values 
    that donot match any element of partiallist in a new list. Returns the new list.'''
    keepers = []
    for i in biglist:
        if i not in partiallist:
            keepers.append(i) 
    return keepers

all_without_above = keep_mis_matches(all_ids, above)
len(all_without_above)


3398

In [32]:
def write_list_to_file(liste, newfile):
    '''Takes as input a list and writes each element to a new file'''
    with open(newfile, 'a') as afile: 
        for i in liste:
            afile.write(i)
            
write_list_to_file(all_without_above, 'try_again')

In [36]:
# all_scoring_below30.py

# input of blastp = fastafile find missing

# interseciton of set below() and above() --> throw_away
# find and remove throw away id file

# make list of all ids reproted in fastinputblastp
# turn fastinputblastp_list into fastinputblastp_set (biggest set)
# fastinputblastp_set - above_set 

def lines_to_list(infile1): 
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(infile1, "r") as rfile:
        content_list = rfile.readlines()
        return content_list
        
below = lines_to_list("id_below_30.hits")  # list of all ids scoring below 30% id
above = lines_to_list('id_above_30.hits')  # list of all ids scoring above and equal to 30% id

def remove_matches(lower, higher):
    '''Takes two lists as input and returns a list that contains
    all values of 'lower' values that are NOT element of 'higher'.'''
    keepers = []          # list holding all ids that have not scored >= 30% with any of the JPRED sequnces
    for i in lower:       
        if i not in higher: 
            keepers.append(i)  # keeps only ids that are not reported in the list "above"
    return keepers

keep = remove_matches(below, above) 

# print(len(keep))
# Have 177 unique sequnces with no match above 30% id with any other sequ in the testing (JPRED) set.

##########################################################
# Making a list of all ids (input IDs of the blastp)
##########################################################

all_ids = lines_to_list("best_of_final_cluster") # generating list of all ids that were in the blastp input
# print(len(all_ids))

def keep_mis_matches(biglist, partiallist):
    '''Stores exclusively biglist values that are not reported in partiallist.
    Returns the biglist with all partiallist matches removed. Keeps all values 
    that donot match any element of partiallist in a new list. Returns the new list.'''
    keepers = []
    for i in biglist:
        if i not in partiallist:
            keepers.append(i) 
    return keepers

all_without_above = keep_mis_matches(all_ids, above)
# len(all_without_above)

def write_list_to_file(liste, newfile):
    '''Takes as input a list and writes each element to a new file'''
    with open(newfile, 'a') as afile: 
        for i in liste:
            afile.write(i)
            
write_list_to_file(all_without_above, 'try_again')


# Randomly Sort and Pick 150 Sequences 

Sometimes not all IDS have an associated PDB file thus I select 160 to be sure.

In [6]:
!cat ids_0-30 | sort -R | head -160 > random.blindset2 # in case some PDB files are not available
!wc -l random.blindset2

     160 random.blindset2


Note that I still have *NOT* removed identical chain IDs:

* The file still contains all letters denoting different chains after the ":"

* printing it to a new file only keeping first letter after the ":"

In [7]:
!tail random.blindset2

5U39:A
6R82:AB
6G3Z:AB
5XJL:2
6EHA:AB
6NTV:ABC
6T8S:AAABBBCCC
4YWN:AB
5FLY:AB
5JVV:AB


In [8]:
!cat random.blindset2 | cut -c-6 > id_and_chain_blindset2

In [9]:
# checking if really no above 30 are in the new list
def lines_to_list(infile1): 
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(infile1, "r") as rfile:
        content_list = rfile.readlines()
        return content_list
        
bigset = lines_to_list("random.blindset2")  # list of all ids scoring below 30% id
above = lines_to_list('id_above_30.hits')  # list of all ids scoring above and equal to 30% id

def remove_matches(l1, l2):
    '''Takes two lists as input and returns a list that contains
    all values of 'lower' values that are NOT element of 'higher'.'''
    keepers = []          # list holding all ids that have not scored >= 30% with any of the JPRED sequnces
    for i in l1:       
        if i not in l2: 
            keepers.append(i)  # keeps only ids that are not reported in the list "above"
    return keepers

keep = remove_matches(bigset, above) 
len(keep)

160

# 6. Download Structures from PDB

For each of the 150 sequences I have to download the pdb structure

In [10]:
!cat random.blindset2 | cut -c-4 > only_id_blindset2

In [ ]:
# for i in
# https://files.rcsb.org/view/6OZJ.pdb

In [ ]:
# 7. Generate DSSP files from selected PDB files

In [11]:
def lines_to_list(infile1): 
    ''' Reads all lines from a file and saves them to a list. '''
    content_list = []
    with open(infile1, "r") as rfile:
        content_list = rfile.readlines()
        return content_list
        
bigset = lines_to_list("only_id_blindset2") # Contain trailing newline char
#removing \n :
def rstrip_each_item(list):
    my_150_PDB = []
    for i in list:
        clean = i.rstrip()
        my_150_PDB.append(clean)
    return my_150_PDB

my_150_PDB = rstrip_each_item(bigset)
print(my_150_PDB)
len(my_150_PDB)

['6LTZ', '6EXX', '5XGA', '5WUJ', '4ZY7', '5F2A', '5D1R', '5UNI', '5U7E', '5ANP', '6HKS', '6J0Y', '6L77', '6KKO', '6GW6', '5LDD', '6K7Q', '4Y0L', '5GNA', '5C8A', '4ZC4', '6EI6', '5UMV', '6OR3', '5U4U', '5XKS', '5GHL', '5XYF', '5N07', '5FB9', '5CEG', '7BWF', '4YTE', '5V0M', '4Y4O', '6SE1', '5UC0', '5WNW', '5IHF', '5T2Y', '5IB0', '6USC', '4UIQ', '4YBB', '6YJ1', '6DHX', '5D6T', '6DN4', '5BP5', '6ISU', '6FSF', '5VOG', '5IR2', '5D71', '5BPX', '5II0', '4Y0O', '5AUN', '5C5Z', '5KWV', '6MDW', '5FQ0', '7BVV', '5AV5', '5FFL', '6OOD', '5KQA', '5DCF', '5GKE', '5ZRY', '5UIV', '4Y4O', '6GBI', '5MC9', '6FWT', '6HSV', '6HFG', '5A88', '5YEI', '6NDR', '5KLC', '6MAB', '6AOZ', '5T2X', '5LTF', '6J19', '6T7O', '6MLX', '5YMX', '7JTL', '5K21', '5CTD', '6R5W', '5EIV', '5HT8', '5DD8', '5D16', '4ZEY', '6VCI', '5Y7W', '4ZLR', '5XVK', '6VK4', '6IA7', '5HJF', '6QLC', '6WK3', '5WOQ', '5YSN', '5Z1N', '5BPU', '5ABR', '4ZKP', '6ON1', '6I9L', '5AZW', '6KOK', '5Y4R', '5CYB', '5A6W', '5U5N', '5BN2', '6UXF', '6K6L', '5WD6',

160

In [ ]:
# import Bio
# from Bio.PDB import PDBList
# '''Selecting structures from PDB'''
# pdbl = PDBList()
# PDBlist2= my_150_PDB
# for i in PDBlist2:
#     pdbl.retrieve_pdb_file(i,pdir='PDB')

In [12]:
!cat only_id_blindset2 | sed -z 's/\n/, /g' > commas2

sed: illegal option -- z
usage: sed script [-Ealn] [-i extension] [file ...]
       sed [-Ealn] [-i extension] [-e script] ... [-f script_file] ... [file ...]


In [1]:
cd blindset_all_PDBs/150_blind_PDBs/


In [ ]:
for i in {1..150}
do
    gunzip *ent.gz
done

# 8. Generating DSSP files from all 150 PDB files

In [ ]:
#!/bin/bash
# Running the DSSP on the extracted PDB files
for i in *.ent
do
        mkdssp -i "$i" -o "./dsspout/$i.dssp"
done

Extracting chain and secondary structure from DSSP files:

* protein chain: ```$3```
* Secondary Structure Summanry ```$4```

In [8]:
!cd /Users/ila/01-Unibo/02_Lab2/project_blindset/blindset_all_PDBs/150_blind_PDBs/dssp_back/


In [9]:
!pwd

/Users/ila/01-Unibo/02_Lab2/project_blindset


In [24]:
# dict holding all possbible 8 SSconformations (keys)
# values are the desired classes that need to be mapped out.
structure_dict = {"H":"H", "G":"H", "I":"H", "B":"E", "E":"E", "T":"C", "S":"C", " ":"C"} 

# try to print only lines after  # 
#  RESIDUE AA STRUCTURE BP1 BP2  ACC     N-H-->O    O-->H-N    N-H-->O    O-->H-N    TCO  KAPPA ALPHA  PHI   PSI    X-CA   Y-CA   Z-CA 


def lines_list(fname):
    with open(fname) as ofile:
        flist = ofile.readlines() # returns list containing each line of the file
        return flist

def relevant_lines(liste, desired_chain):
    relevant = False # boolean variable ###################################################################
#     desired_chain = "A" # change to load from id_and_chain_blindset2
    raw_ccstring = ''  ####################################################################################
    three_classes = ''
    aa_string = ''
    for line in liste:
        if '#' in line: # find last line before relevant output
            relevant =True   # flips rel to true - so the folowing lines are saved
            continue
        if relevant:
            if line[11] == desired_chain:
                raw_ccstring += line[16]
                aa_string += line[13]           
    return raw_ccstring, aa_string

def raw_to_threclasses(rawstring):
        threeclasses = ''
        for letter in rawstring:
                threeclasses += structure_dict[letter]
        return threeclasses

id_chain = lines_list('id_and_chain_blindset2')

for id in id_chain:
    fields_lists = id.split(':') # list contains ID and chain
    fname = "pdb"+fields_lists[0].lower()+".ent.dssp"
    chain = fields_lists[1]
    
    
mylist = lines_list("/Users/ila/01-Unibo/02_Lab2/project_blindset/blindset_all_PDBs/150_blind_PDBs/dssp_back/pdb4zy7.ent.dssp")
raw, aa = relevant_lines(mylist)

three_class_string = raw_to_threclasses(raw)

print('>4uiq: chain??????')
print(aa)
print('The three classes:')
print(three_class_string)

# read file and 
# 4Y4O:14 
# 4Y4O:18 
# 5XJL:2

# id_and_chain_blindset2 contains all the IDs and corresponding chains
# need to wirte a script that
# takes id_and chain as input (list) and adds them to the header of each corresponding protein
# pdb input file
# Program needs to use desired chain as input from that file too


>4uiq: chain??????
DPAKTLEAVSAVADWLRDPQRESPARAQLAEAVRLTARTLAAVAPGASVEVRVPPFVAVQCISGPKHTRGTPPNVVETDARTWLLLATGLLDIADAGASVQMSGSRAAEVAHWLPVVRI
The three classes:
CHHHHHHHHHCCHHHHHCCCCCCCCHHHHHHHHHHHHHHHHHHCCCCCEEEEECCCEEEEECCCCCCCCCCCCEEEEECHHHHHHHHHCCCCHHHCHHHEEEECCCHHHHHHHCCCCCC


In [ ]:
 scp -i ~/.ssh/id_rsa.pub ./150_blind_PDBs/* proj:~/lb2-2020-project-englander/150_blind_PDBs/

# Extracting Desired Chain From Each DSSP file


In [38]:
!ls id_and_chain_blindset2

id_and_chain_blindset2


In [39]:
!cat id_and_chain_blindset2

6LTZ:A
6EXX:A
5XGA:A
5WUJ:B
4ZY7:A
5F2A:A
5D1R:A
5UNI:A
5U7E:A
5ANP:A
6HKS:A
6J0Y:C
6L77:A
6KKO:A
6GW6:B
5LDD:B
6K7Q:A
4Y0L:A
5GNA:B
5C8A:A
4ZC4:A
6EI6:A
5UMV:A
6OR3:A
5U4U:A
5XKS:A
5GHL:A
5XYF:A
5N07:A
5FB9:A
5CEG:A
7BWF:B
4YTE:A
5V0M:A
4Y4O:14
6SE1:A
5UC0:A
5WNW:A
5IHF:A
5T2Y:A
5IB0:A
6USC:A
4UIQ:A
4YBB:C
6YJ1:A
6DHX:A
5D6T:A
6DN4:A
5BP5:C
6ISU:A
6FSF:A
5VOG:A
5IR2:A
5D71:A
5BPX:A
5II0:A
4Y0O:A
5AUN:A
5C5Z:A
5KWV:A
6MDW:A
5FQ0:A
7BVV:A
5AV5:A
5FFL:A
6OOD:A
5KQA:A
5DCF:A
5GKE:A
5ZRY:A
5UIV:A
4Y4O:18
6GBI:A
5MC9:B
6FWT:A
6HSV:A
6HFG:B
5A88:A
5YEI:B
6NDR:A
5KLC:A
6MAB:A
6AOZ:A
5T2X:A
5LTF:A
6J19:B
6T7O:A
6MLX:A
5YMX:A
7JTL:A
5K21:A
5CTD:C
6R5W:A
5EIV:A
5HT8:A
5DD8:A
5D16:A
4ZEY:A
6VCI:A
5Y7W:A
4ZLR:A
5XVK:A
6VK4:D
6IA7:A
5HJF:A
6QLC:A
6WK3:A
5WOQ:A
5YSN:B
5Z1N:A
5BPU:A
5ABR:A
4ZKP:A
6ON1:A
6I9L:A
5AZW:A
6KOK:A
5Y4R:C
5CYB:A
5A6W:C
5U5N:A
5BN2:A
6UXF:A
6K6L:A
5WD6:A
6OVI:A
5V2I:A
6IQO:A
6MD3:A
5JSN:B
5JWO:A
6Q7N:A
5NL9:A
6Q8J:A
5WD8:A
6CEQ:A
5BPK:C
5BXQ:A
6OKM:R
5MMH:A
6H9E:A
5F1S:A
6SLK